In [2]:
from xml.etree.ElementTree import parse
# 不处理test，拆分train
path='train'
data1 = parse(path+'.xml')
root=data1.getroot()
sentiment2id = {'negative':'1','neutral':'2','positive':'3','conflict':'4'}
def get_text(root):
    with open(path+'.tsv','w') as f:
        f.write('labels\tkey_words\treview\n')
        for sentence in root.iter('sentence'):
            text = sentence.find('text').text

            sentiments = []
            aspects_cats = []
            aspect_cat = sentence.find('aspectCategories')
            for each in aspect_cat.iter('aspectCategories'):
                each_ = each.findall('aspectCategory')
                for each2 in each_:
                    aspects_cats.append( each2.attrib['category'])
                    try:
                        sentiments.append(each2.attrib['polarity']) if each2.attrib['polarity'] in ['positive','negative','neutral','conflict'] else None
                    except:
                        sentiments.append('neutral')
            aspectTerms = []
            aspects = sentence.find('aspectTerms')
            try:
                for each in aspects.iter('aspectTerms'):
                    aspectTerms.append( each.find('aspectTerm').attrib['term'])
            except:
                aspectTerms.append('NEGATIVE')

            for i in range(0,len(sentiments)):

                try:
                    f.write(sentiment2id[sentiments[i]]+aspects_cats[i]+'\t'+aspectTerms[i]+'\t'+"\""+text+"\""+'\n')
                except:
                    f.write(sentiment2id[sentiments[i]]+aspects_cats[i]+'\t'+aspectTerms[0]+'\t'+"\""+text+"\""+'\n')
print(get_text(root))


None


In [3]:
import pandas as pd
import numpy as np
ALL_cats=[]
path1='test'
path2='train' if path1=='test' else 'test'
data1=pd.read_csv(path1+'.tsv',delimiter='\t')
data1.columns=['1','2','3']
data1['senti']=data1['1'].apply(lambda x:x[0])
data1['CAT']=data1['1'].apply(lambda x:x[1:])
data1['KEYWORDS']=data1['2']
data1['review']=data1['3']
data1=data1[['senti','CAT','KEYWORDS','review']]
data1_=pd.DataFrame(columns=['CAT','KEYWORDS','review'])
for i in range(len(data1)):
    review=data1['review'][i]
    cat=[data1['CAT'][i]]
    ALL_cats.extend(cat)
    senti=data1['senti'][i]
    keywords=data1['KEYWORDS'][i] if type(data1['KEYWORDS'][i])==str else 'NEG'
    for j in cat:
        data1_=data1_.append(
            pd.DataFrame(
                [[j,str(senti)+' '+keywords,review]],
                columns=['CAT','KEYWORDS','review']
            )
        )


data2=pd.read_csv(path2+'.tsv',delimiter='\t',encoding='gbk')
data2.columns=['1','2','3']
data2['senti']=data2['1'].apply(lambda x:x[0])
data2['CAT']=data2['1'].apply(lambda x:x[1:])
data2['KEYWORDS']=data2['2']
data2['review']=data2['3']
data2=data2[['senti','CAT','KEYWORDS','review']]
data2_=pd.DataFrame(columns=['CAT','KEYWORDS','review'])
for i in range(len(data2)):
    review=data2['review'][i]
    cat=[data2['CAT'][i]]
    ALL_cats.extend(cat)
    senti=data2['senti'][i]
    keywords=data2['KEYWORDS'][i] if type(data2['KEYWORDS'][i])==str else 'NEG'
    for j in cat:
        data2_=data2_.append(
            pd.DataFrame(
                [[j,str(senti)+' '+keywords,review]],
                columns=['CAT','KEYWORDS','review']
            )
        )

classes=list(set(ALL_cats))
# 排序
for each in classes:
    try:
        print(each)
    except:
        pass

with open('label.txt','w') as f:
    for each in classes:
        f.write(each+'\n')
data1_.to_csv('关键词抽取结果_'+path1+'.csv',index=False,header=['labels','key_words','review'])
data2_.to_csv('关键词抽取结果_'+path2+'.csv',index=False,header=['labels','key_words','review'],encoding='utf-8')

C:\Users\93461\AppData\Local\Temp\ipykernel_41540\2312393946.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1_=data1_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_41540\2312393946.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1_=data1_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_41540\2312393946.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1_=data1_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_41540\2312393946.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1_=data1_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_41540\2312393946.py:21: FutureWarning: The frame.append 

anecdotes/miscellaneous
ambience
food
service
price


C:\Users\93461\AppData\Local\Temp\ipykernel_41540\2312393946.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data2_=data2_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_41540\2312393946.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data2_=data2_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_41540\2312393946.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data2_=data2_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_41540\2312393946.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data2_=data2_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_41540\2312393946.py:44: FutureWarning: The frame.append 

In [29]:
# 数据增强
import pandas as pd
data = pd.read_csv('train.tsv',sep='\t')
data['index2']=data['labels'].apply(lambda x:x[0])
data1  = data[data['index2']=='4']

data=data.append([data1]*3,ignore_index=True)
data =data[['labels','key_words','review']]
data.to_csv('train_aug.tsv',sep='\t',index=False,header=['labels','key_words','review'])

C:\Users\93461\AppData\Local\Temp\ipykernel_2100\3306701093.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append([data1]*3,ignore_index=True)
